In [ ]:
!pip install langchain faiss-cpu sentence-transformers openai langchain_community
# !pip install langchain faiss-cpu sentence-transformers huggingface_hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
import os

# OpenRouter key
os.environ["OPENAI_API_KEY"] = "-"
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"

In [ ]:
from langchain.chat_models import ChatOpenAI

# Default LLM (safer for legal/e-commerce queries)
llm_default = ChatOpenAI(
    model="shisa-ai/shisa-v2-llama3.3-70b:free",
    temperature=0,
    max_tokens=512
)

# Fallback LLM (Meta LLaMA 3.3 - 8B)
llm_fallback = ChatOpenAI(
    model="meta-llama/llama-3.3-8b-instruct:free",
    temperature=0,
    max_tokens=512
)




/tmp/ipython-input-185725627.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm_default = ChatOpenAI(


In [ ]:
import glob

doc_files = glob.glob("docs/**/*.md", recursive=True)

all_texts = []
for file in doc_files:
    with open(file, "r", encoding="utf-8") as f:
        text = f.read().strip()
        if text:
            all_texts.append({
                "filename": os.path.basename(file),
                "filepath": file,
                "content": text
            })

print(f"[✓] Loaded {len(all_texts)} .md files.")


[✓] Loaded 5 .md files.


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100
)

text_chunks = []
for doc in all_texts:
    chunks = splitter.split_text(doc["content"])
    for i, chunk in enumerate(chunks):
        text_chunks.append({
            "content": chunk,
            "source": doc["filename"],
            "filepath": doc["filepath"],
            "chunk_id": i
        })

print(f"[✓] Created {len(text_chunks)} chunks from all docs.")


[✓] Created 554 chunks from all docs.


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document

embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

docs = [
    Document(
        page_content=chunk["content"],
        metadata={
            "source": chunk["source"],
            "filepath": chunk["filepath"],
            "chunk_id": chunk["chunk_id"]
        }
    )
    for chunk in text_chunks
]

db = FAISS.from_documents(docs, embedder)

print("[✓] FAISS index built.")


/tmp/ipython-input-4113529480.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or da

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[✓] FAISS index built.


In [ ]:
def ask(query, k=5):
    rel_docs = db.similarity_search(query, k=k)

    context = "\n\n---\n\n".join(
        [f"[Source: {doc.metadata['source']} | Chunk: {doc.metadata['chunk_id']}]\n{doc.page_content}"
         for doc in rel_docs]
    )

    prompt = f"""
You are an e-commerce legal policy assistant.
Answer ONLY using the provided documents.
Do NOT hallucinate.
If info is missing, say so.
Maintain professional tone.

### User Question:
{query}

### Retrieved Context:
{context}

### Instructions:
1. Interpret the question.
2. Extract relevant policies from documents.
3. Reason step-by-step.
4. Give final answer.
"""

    try:
        # Try default LLM first
        return llm_default.predict(prompt)
    except Exception as e:
        print("⚠️ Default LLM blocked query. Using fallback LLM...")
        return llm_fallback.predict(prompt)





In [ ]:
print(ask("What are the customer rights if a delivery is delayed?, answer in layman"))


**Interpretation of the Question:**  
The user asks about customer rights when a delivery is delayed in an e-commerce transaction, seeking a layman's explanation.  

**Relevant Policies Extracted:**  
1. **Consumer_protection_2020.md (Chunk 27 & 28) & ConsumerProtection_ECommerceRules2020.md (Chunk 25 & 34):**  
   - Sellers (both marketplace and inventory e-commerce entities) must not refuse a refund if goods/services are delivered late from the stated schedule.  
   - Exception: Delay due to **force majeure** (unforeseeable circumstances like natural disasters) is excluded.  

2. **Consumer_protection_2020.md (Chunk 36):**  
   - Inventory e-commerce entities guaranteeing authenticity bear liability for defects, including late delivery (unless force majeure applies).  

**Step-by-Step Reasoning:**  
- **Right to Refund:** If delivery is late (and not due to force majeure), the customer can demand a refund.  
- **Grievance Redressal:** Customers can file complaints with the seller’s g

In [ ]:
response = ask("I was offered 10% discount, but now there is no discount. What are my consumer rights?")
print(response)



### Interpretation of the Question:  
The user was initially offered a 10% discount but is no longer receiving it. They are inquiring about their consumer rights in this situation.  

### Extracted Relevant Policies:  
1. **Consumer_protection_2020.md | Chunk: 18**:  
   - Prohibits sellers from discriminating between consumers or making arbitrary classifications affecting their rights.  
   - Requires justification for price changes based on relevant circumstances.  

2. **ConsumerProtection_ECommerceRules2020.md | Chunk: 25**:  
   - Sellers must refund or take back goods/services if they do not match advertised features or are delivered late.  

3. **ConsumerRights2019.md | Chunk: 35**:  
   - Mentions significant changes in consumer rights (though specifics are not provided in the given chunk).  

### Step-by-Step Reasoning:  
1. **Discount as an Advertised Feature**:  
   - If the 10% discount was explicitly advertised or agreed upon at the time of purchase, removing it may violat

In [ ]:
print(ask("What are the rules for returning defective products under Indian e-commerce regulations?"))

### Interpretation of the Question:  
The user seeks the rules governing returns of defective products under Indian e-commerce regulations.  

### Extracted Relevant Policies:  
1. **ConsumerProtection_ECommerceRules2020.md (Chunk 34)**:  
   - Rule (4) states that no inventory e-commerce entity can refuse to take back defective goods or refuse a refund if the goods are defective, deficient, spurious, or not as advertised.  
   - Exception: Late delivery due to force majeure is excluded.  

2. **ConsumerRights2019.md (Chunk 195 & 199)**:  
   - Defines product liability actions under the Consumer Protection Act, 2019.  
   - A product manufacturer/seller is liable for defects in manufacturing, design, deviation from specs, or inadequate warnings.  

### Step-by-Step Reasoning:  
1. **Legal Basis**: The Consumer Protection Act, 2019, and the E-Commerce Rules, 2020, govern defective product returns.  
2. **Return Policy**: Inventory e-commerce entities must accept returns for defective g

In [ ]:
print(ask("What happens if an e-commerce platform fails to deliver a product within the promised timeframe?"))

/tmp/ipython-input-584389209.py:31: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return llm_default.predict(prompt)


**Interpretation of the Question:**  
The user asks what happens if an e-commerce platform fails to deliver a product within the promised timeframe.  

**Relevant Policies Extracted:**  
1. **Consumer_protection_2020.md | Chunk: 27 & ConsumerProtection_ECommerceRules2020.md | Chunk: 25:**  
   - Sellers cannot refuse to take back goods or refuse a refund if goods/services are delivered late from the stated delivery schedule.  
   - Exception: Late delivery due to *force majeure* (unforeseeable circumstances) may exempt liability.  

2. **Consumer_protection_2020.md | Chunk: 36 & ConsumerProtection_ECommerceRules2020.md | Chunk: 34:**  
   - Inventory e-commerce entities must refund or take back goods if delivered late (unless *force majeure* applies).  

**Step-by-Step Reasoning:**  
- The policy requires sellers/platforms to refund or take back goods if delivery is late (unless due to *force majeure*).  
- No additional penalties or specific actions (e.g., compensation) are mentioned 